In [6]:
from faker import Faker
import random
import pandas as pd
fake = Faker()

In [7]:
def generate_custom_population(size=10000):
    return pd.DataFrame({
        'name': [fake.name() for _ in range(size)],
        'age': [random.randint(18, 90) for _ in range(size)],
        'state': [fake.state() for _ in range(size)],
        'gender': [random.choice(['Male', 'Female']) for _ in range(size)],
        'address': [fake.address() for _ in range(size)],
    })

In [8]:
custom_population = generate_custom_population(10000)

In [10]:
custom_population['address'] = custom_population['address'].str.replace('\n', ' ', regex=False)

In [11]:
custom_population.head()

,name,age,state,gender,address
0,Charles Brady,62,Oklahoma,Female,203 Christopher Mews Suite 416 South Natalieha...
1,Eric Banks,69,North Dakota,Male,"4953 Wright Forks Suite 254 West Tammyborough,..."
2,Cody Jenkins,78,Virginia,Male,"68406 Alexander Knolls Suite 470 Williamsbury,..."
3,Mary Garcia,66,Kentucky,Female,"123 Jones Trail North Emilyfort, MN 30882"
4,Sonia Gibson,25,Nebraska,Male,"550 Martin Forges Apt. 168 Richardsonland, NJ ..."
